In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as T

In [ ]:
mnist_data_train = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=T.ToTensor()
)

mnist_data_test = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=T.ToTensor()
)

In [ ]:
train_dataloader = DataLoader(mnist_data_train, batch_size=64, shuffle=True)
test_dataloader = DataLoader(mnist_data_test, batch_size=64, shuffle=True)

In [ ]:
class NeuraalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.layer1 = nn.Linear(16*5*5, 128) 
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 10)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.flatten(x)
        x = self.activation(self.layer1(x))
        x = self.activation(self.layer2(x))
        x = self.layer3(x)
        return x

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = NeuraalNetwork().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
def train(loader, model, criterion, optimizer):
    total_loss = 0.0
    total_samples = 0
    correct_predictions = 0
    
    for X, y in loader:
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred, y)
        batch_size = X.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size
        
        _, predicted = torch.max(y_pred, 1)
        correct_predictions += (predicted == y).sum().item() 

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples * 100
    print(f"Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

In [ ]:
def test(loader, model, criterion):
    size = len(loader.dataset)
    num_batches = len(loader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Loss: {test_loss:>8f}")

In [ ]:
epochs = 25
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train(train_dataloader, model, criterion, optimizer)
    test(test_dataloader, model, criterion)


In [ ]:
torch.save(model.state_dict(), "model_cnn.pth")
print("Modèle sauvegardé")